In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.8 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("/content/drive/MyDrive/jigsaw-toxic-comment-classification-challenge (1)/train.csv/train.csv")

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [7]:
data=data[['comment_text','toxic']]

In [8]:
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
data=data[0:1000]

In [10]:
data.shape

(1000, 2)

In [11]:
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch

In [13]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer,BertForSequenceClassification

In [14]:
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
model=model.to('cuda')

In [17]:
X = list(data["comment_text"]) # tokenizer accept the list of strings
y = list(data["toxic"]) 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [18]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None): # take the tokenized data and labels 
    self.encodings=encodings  # tokenized text
    self.labels=labels

  def __getitem__(self,idx): #it will return the example/data from that index
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[idx])
      return item

  def __len__(self): # return the length of inputs_ids(number of example we have)
        return len(self.encodings["input_ids"])


In [19]:
train_data=Dataset(X_train_tokenized,y_train)
test_data=Dataset(X_val_tokenized,y_val)

In [ ]:
train_data[5]

In [21]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [22]:
#define Trainer
args=TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8  #how many datapoints you want to send in each batch
)
trainer=Trainer(model=model,args=args,train_dataset=train_data,eval_dataset=test_data,compute_metrics=compute_metrics)

In [23]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 800
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.26358470916748045, metrics={'train_runtime': 71.3917, 'train_samples_per_second': 11.206, 'train_steps_per_second': 1.401, 'total_flos': 210488844288000.0, 'train_loss': 0.26358470916748045, 'epoch': 1.0})

In [24]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.14848218858242035,
 'eval_accuracy': 0.95,
 'eval_precision': 0.8235294117647058,
 'eval_recall': 0.6666666666666666,
 'eval_f1': 0.7368421052631577,
 'eval_runtime': 6.015,
 'eval_samples_per_second': 33.25,
 'eval_steps_per_second': 4.156,
 'epoch': 1.0}

In [25]:
text="fuck off"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.3087,  1.1394]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [26]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [27]:
print(predictions)

tensor([[0.0796, 0.9204]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [28]:
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.07957448, 0.9204255 ]], dtype=float32)

In [29]:
trainer.save_model('CustomModel')

Saving model checkpoint to CustomModel
Configuration saved in CustomModel/config.json
Model weights saved in CustomModel/pytorch_model.bin


In [30]:
model2=BertForSequenceClassification.from_pretrained('/content/CustomModel')

loading configuration file /content/CustomModel/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /content/CustomModel/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClass

In [31]:
model2=model2.to('cuda')

In [32]:
text="Go to Hell"
input=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')
output=model2(**input)

In [33]:
predictions = torch.nn.functional.softmax(output.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.61084354, 0.38915646]], dtype=float32)

In [34]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import torch 
from transformers import BertTokenizer, BertForSequenceClassification

In [38]:
from huggingface_hub import notebook_login

In [39]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [40]:
model2.push_to_hub('VegetaSama/FineTuneBert')

Configuration saved in /tmp/tmpa5n60c8x/config.json
Model weights saved in /tmp/tmpa5n60c8x/pytorch_model.bin
Uploading the following files to VegetaSama/FineTuneBert: config.json,pytorch_model.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/VegetaSama/FineTuneBert/commit/784bddd1b225a655959d6bab79c16a711e99953a', commit_message='Upload BertForSequenceClassification', commit_description='', oid='784bddd1b225a655959d6bab79c16a711e99953a', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
model=BertForSequenceClassification.from_pretrained('VegetaSama/FineTuneBert')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--VegetaSama--FineTuneBert/snapshots/784bddd1b225a655959d6bab79c16a711e99953a/config.json
Model config BertConfig {
  "_name_or_path": "/content/CustomModel",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--VegetaSama--FineTuneBert/snapshots/784bddd1b225a655959d6bab79c16a711e99953a/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at VegetaSama/FineTuneBert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
